In [1]:
import sql
import pandas as pd
from functools import reduce

In [2]:
q='''select name, count(*) as total from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        group by name
        order by name'''
ta = pd.DataFrame(sql.execute(q)) 
q='''select name, count(*) as memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory =1 
        group by name
        order by name'''
tma = pd.DataFrame(sql.execute(q)) 
tma['memory (%)']= round((tma.memory / ta.total) * 100,2)
q='''select name, count(*) as non_memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory is null
        group by name
        order by name'''
tnma = pd.DataFrame(sql.execute(q)) 
tnma['non_memory (%)']= round((tnma.non_memory / ta.total) * 100,2)
df= reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df['memory (%)']=df['memory (%)'].astype(str)+"%"
df['non_memory (%)']=df['non_memory (%)'].astype(str)+"%"
df

,name,total,memory,memory (%),non_memory,non_memory (%)
0,Firefox,12945,5407,41.77%,7538,58.23%
1,Kodi,2393,640,26.74%,1753,73.26%
2,Linux,19519,7600,38.94%,11919,61.06%
3,ovirt-engine,2989,1273,42.59%,1716,57.41%
4,Samba,4507,1734,38.47%,2773,61.53%


In [21]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and actionability = 1
and me.memory =1
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)   
df.e=df.e.astype(str)+'%'
df.a=df.a.astype(str)+'%'
df.b=df.b.astype(str)+'%'
df.rename(columns={'total':'total memory', 'e':'eliminated memory', 'a':'actionable memory','b':'triaged_bug memory'}, inplace=True)
memory = df        
memory                                                 

,total memory,eliminated memory,actionable memory,triaged_bug memory
name,,,,
Firefox,5407,77.23%,49.81%,12.39%
Kodi,640,69.22%,54.06%,24.53%
Linux,7600,73.38%,43.01%,2.89%
Samba,1734,78.43%,39.1%,3.75%
ovirt-engine,1273,35.98%,27.02%,2.51%


In [22]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and actionability = 1
and me.memory is null
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)   
df.e=df.e.astype(str)+'%'
df.a=df.a.astype(str)+'%'
df.b=df.b.astype(str)+'%'
df.rename(columns={'total':'total non_memory', 'e':'eliminated non_memory', 'a':'actionable non_memory','b':'triaged_bug non_memory'}, inplace=True)
non_memory = df        
non_memory     

,total non_memory,eliminated non_memory,actionable non_memory,triaged_bug non_memory
name,,,,
Firefox,7538,70.92%,46.43%,5.2%
Kodi,1753,67.08%,50.43%,12.09%
Linux,11919,65.16%,39.51%,3.55%
Samba,2773,72.95%,25.03%,1.55%
ovirt-engine,1716,47.9%,32.69%,2.62%


In [23]:
df=reduce(lambda x,y: pd.merge(x,y,on='name'),[memory,non_memory])
df=df[['total memory', 'total non_memory', 'eliminated memory', 'eliminated non_memory',
    'actionable memory','actionable non_memory','triaged_bug memory','triaged_bug non_memory']]
df

,total memory,total non_memory,eliminated memory,eliminated non_memory,actionable memory,actionable non_memory,triaged_bug memory,triaged_bug non_memory
name,,,,,,,,
Firefox,5407,7538,77.23%,70.92%,49.81%,46.43%,12.39%,5.2%
Kodi,640,1753,69.22%,67.08%,54.06%,50.43%,24.53%,12.09%
Linux,7600,11919,73.38%,65.16%,43.01%,39.51%,2.89%,3.55%
Samba,1734,2773,78.43%,72.95%,39.1%,25.03%,3.75%,1.55%
ovirt-engine,1273,1716,35.98%,47.9%,27.02%,32.69%,2.51%,2.62%
